In [1]:
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
import numpy as np
import datetime

In [4]:
def get_soup():
    df = pd.DataFrame(columns=['job_id', 'company_id', 'location', 'designation', 'company_name', 'job_url'])
    for i in range(1, 100):
        url = "https://www.naukri.com/jobs-in-india-" + str(i)
        driver = webdriver.Chrome()
        driver.get(url)
        
        time.sleep(10)
        
        soup = BeautifulSoup(driver.page_source, 'html5lib')
        
        # Open a new window
        driver.execute_script("window.open('about:blank', '_blank');")

        # Switch to the new window
        driver.switch_to.window(driver.window_handles[1])

        try:
            # Code that interacts with the new window (e.g., scraping data)
            # Make sure to interact with elements in the new window here
            # Example: job_url_new = driver.find_element_by_xpath('xpath_of_element').get_attribute('href')
            pass  # Placeholder for the actual code
            
        except NoSuchWindowException as e:
            print("Window closed prematurely:", str(e))
            # Handle the exception appropriately, e.g., break out of the loop or continue with the next iteration
        
        driver.close()  # Close the new window
        
        # Switch back to the original window
        driver.switch_to.window(driver.window_handles[0])

        # Continue with the rest of your code (e.g., scraping data from the original window)
        job_list = soup.find(class_='list')
        job_elems = job_list.find_all('article',class_="jobTuple")
        for job in job_elems:
            job_url = job.find('a',class_='title ellipsis').get('href')
            designation = job.find('a',class_='title ellipsis').text
            company_name = job.find('a',class_='subTitle ellipsis fleft').text
            # rating_span = job.find('span',class_='starRating fleft dot')
            job_location_element = job.find('span', class_='ellipsis fleft locWdth')
            if job_location_element:
                job_location = job_location_element.text
            else:
                job_location = "Location not found"
            job_id = job_url.split("-")[-1]
            company_id = job.find('a',class_='subTitle ellipsis fleft').get('href').split("-")[-1]
            df1= pd.DataFrame({"job_id":[job_id], "company_id":[company_id], "location":[job_location],"designation":[designation],"company_name":[company_name],"job_url":[job_url]})
            df = pd.concat([df,df1],ignore_index = True)

    return df

df = get_soup()


In [8]:
def make_url_link(x):
    l = x.split()
    link = ""
    for i in l:
        link = link + i.lower() + "-"
    return "https://www.ambitionbox.com/overview/" + link + "overview"
        
df["overview_link"] = df["company_name"].apply(make_url_link)


In [7]:
df2 = pd.DataFrame(columns = ['overview_link','founded_year', 'india_employee_count', 'global_employee_count'])
d = {}
for url in df.overview_link:
     
    # using now() to get current time
    # print(datetime.datetime.now())
    values = {}
    try:
        if url in d:
            values = d[url]
        else:
            driver = webdriver.Chrome()
            driver.get(url)
            
            time.sleep(5)
            
            soup = BeautifulSoup(driver.page_source,'html5lib')
            
            driver.close()
            job_list = soup.find(class_='aboutTable')
            job_elems = job_list.find_all('li', class_='aboutItem')
            
            for li in job_elems:
                name_element = li.find('p', class_='aboutItem__name')
                value_element = li.find('p', class_='textItem__val aboutItem__value')
                
                if name_element and value_element:
                    name = name_element.get_text(strip=True)
                    value = value_element.get_text(strip=True)
                    values[name] = value
                    
            if 'Founded in' not in values:
                values['Founded in'] = np.nan
            elif 'Global Employee count' not in values:
                values['Global Employee count'] = np.nan
            elif 'India Employee count' not in values:
                values['India Employee count'] = np.nan
            d[url] = values
        
        df1 = pd.DataFrame({"overview_link": [url],'founded_year': [values['Founded in']], 'india_employee_count': [values['India Employee count']], 'global_employee_count': [values['Global Employee count']]})
        df2 = pd.concat([df2,df1])    
    except:
        df1 = pd.DataFrame({"overview_link": [url],'founded_year': [np.nan], 'india_employee_count': [np.nan], 'global_employee_count': [np.nan]})
        df2 = pd.concat([df2,df1])
    

                                                                                                                

In [9]:
df2.drop("overview_link",axis = 1, inplace = True)
df2.reset_index(drop = True, inplace = True)
    
df = pd.concat([df,df2],axis = 1)
df

,job_id,company_id,location,designation,company_name,job_url,overview_link,founded_year,india_employee_count,global_employee_count
0,070923501018,19589,"Kolkata, Mumbai, New Delhi, Hyderabad/Secunder...",Associate - Inventory Accounting,Ford,https://www.naukri.com/job-listings-associate-...,https://www.ambitionbox.com/overview/ford-over...,1903 (120 yrs old),5k-10k,1 Lakh+
1,270523906122?src=jobsearchDesk&sid=16941696232...,7682,"Kolkata, West Bengal, Mumbai, Maharashtra, Hyd...",CN - Comms & Media - Network Assurance - Senio...,Accenture,https://www.naukri.com/job-listings-cn-comms-m...,https://www.ambitionbox.com/overview/accenture...,1989 (34 yrs old),1 Lakh+,1 Lakh+
2,230920500882,4571516,"Kolkata, Mumbai, New Delhi, Hyderabad/Secunder...",Software Developer/Trainer,Thrive Info Tech,https://www.naukri.com/job-listings-software-d...,https://www.ambitionbox.com/overview/thrive-in...,NaN,NaN,NaN
3,210123500752,1295512,"Kolkata, Mumbai, New Delhi, Hyderabad/Secunder...",Intern,Muthoot Finance,https://www.naukri.com/job-listings-intern-mut...,https://www.ambitionbox.com/overview/muthoot-f...,1939 (84 yrs old),10k-50k,10k-50k
4,180722500200,1295512,"Kolkata, Mumbai, New Delhi, Hyderabad/Secunder...",MALE INTERN - ERODE REGION,Muthoot Finance,https://www.naukri.com/job-listings-male-inter...,https://www.ambitionbox.com/overview/muthoot-f...,1939 (84 yrs old),10k-50k,10k-50k
...,...,...,...,...,...,...,...,...,...,...
1975,060923910646,7682,Bangalore/Bengaluru,MuleSoft Anypoint Platform Application Developer,Accenture,https://www.naukri.com/job-listings-mulesoft-a...,https://www.ambitionbox.com/overview/accenture...,1989 (34 yrs old),1 Lakh+,1 Lakh+
1976,060923910639,7682,Bangalore/Bengaluru,Servicenow Tools Administration Program/Projec...,Accenture,https://www.naukri.com/job-listings-servicenow...,https://www.ambitionbox.com/overview/accenture...,1989 (34 yrs old),1 Lakh+,1 Lakh+
1977,060923910635,7682,Bangalore/Bengaluru,SAP PP Production Planning & Control Discrete ...,Accenture,https://www.naukri.com/job-listings-sap-pp-pro...,https://www.ambitionbox.com/overview/accenture...,1989 (34 yrs old),1 Lakh+,1 Lakh+
1978,060923910627,7682,Bangalore/Bengaluru,SAP Purchasing (MM PO) Technology Educator,Accenture,https://www.naukri.com/job-listings-sap-purcha...,https://www.ambitionbox.com/overview/accenture...,1989 (34 yrs old),1 Lakh+,1 Lakh+


In [12]:
col = ['Role:', 'Industry Type:', 'Department:', 'Employment Type:','Role Category:']
df5 = pd.DataFrame(columns = ['Role:', 'Industry Type:', 'Department:', 'Employment Type:','Role Category:', 'description', 'skills_required'])
for url in df.job_url:
    details = {}
    
    try:
        driver = webdriver.Chrome()
        driver.get(url)
        
        time.sleep(5)
        
        soup = BeautifulSoup(driver.page_source,'html5lib')
        
        driver.close()
        
        job_list = soup.find(class_='styles_key-skill__GIPn_')
        job_elems = job_list.find_all('span')
        skills_required = []
        for skill in job_elems:
            skills_required.append(skill.text)
        
        description_section = soup.find('h2', text='Description')
        if description_section is None:
            description_section = soup.find('h2', text='Job description')
        description_list_items = description_section.find_next('ul').find_all('li')
        description = [item.text.strip() for item in description_list_items]
        
        other_details_section = soup.find('div', class_='styles_other-details__oEN4O')
        for label_element in other_details_section.find_all('label'):
            label = label_element.text.strip()
            value = label_element.find_next('span').text.strip()
            details[label] = [value]
        # details["description"] = description
        # details["skills_required"] = skills_required

        for i in col:
            if i not in details:
                details[i] = np.nan
        df4 = pd.DataFrame(details)
        df4['description'] = ",".join(description)
        df4['skills_required'] = ",".join(skills_required)
        df5 = pd.concat([df5,df4])
    except:
        for i in df5.columns:
            details[i] = [np.nan]
        df4 = pd.DataFrame(details)
        df5 = pd.concat([df5,df4])
        
df5







,Role:,Industry Type:,Department:,Employment Type:,Role Category:,description,skills_required
0,"Finance Executive,","Auto Components,","Finance & Accounting,","Full Time, Permanent",Finance,"Good reasoning capability, innovative mindset,...","inventory accounting,Management"
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,"Back End Developer,","Management Consulting,","Engineering - Software & QA,","Full Time, Permanent",Software Development,Teaching various computer languages and specia...,"Administration,Computer languages,C++,Analytic..."
0,"Business Development Executive (BDE),","Banking,","Sales & Business Development,","Full Time, Permanent",BD / Pre Sales,Canvassing and sourcing of New Customers to in...,"Sales,Financial sector,Accounting,Gold loan,Cu..."
0,"Customer Service,","Banking,","Customer Success, Service & Operations,","Full Time, Permanent","Customer Success, Service & Operations - Other",Handling Walk-ins Enquiries Supporting Custome...,"Backend,Intern,Client servicing,Management"
...,...,...,...,...,...,...,...
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
finaldf = pd.concat([df,df5.reset_index(drop = True)],axis = 1)
finaldf

,job_id,company_id,location,designation,company_name,job_url,overview_link,founded_year,india_employee_count,global_employee_count,Role:,Industry Type:,Department:,Employment Type:,Role Category:,description,skills_required
0,070923501018,19589,"Kolkata, Mumbai, New Delhi, Hyderabad/Secunder...",Associate - Inventory Accounting,Ford,https://www.naukri.com/job-listings-associate-...,https://www.ambitionbox.com/overview/ford-over...,1903 (120 yrs old),5k-10k,1 Lakh+,"Finance Executive,","Auto Components,","Finance & Accounting,","Full Time, Permanent",Finance,"Good reasoning capability, innovative mindset,...","inventory accounting,Management"
1,270523906122?src=jobsearchDesk&sid=16941696232...,7682,"Kolkata, West Bengal, Mumbai, Maharashtra, Hyd...",CN - Comms & Media - Network Assurance - Senio...,Accenture,https://www.naukri.com/job-listings-cn-comms-m...,https://www.ambitionbox.com/overview/accenture...,1989 (34 yrs old),1 Lakh+,1 Lakh+,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,230920500882,4571516,"Kolkata, Mumbai, New Delhi, Hyderabad/Secunder...",Software Developer/Trainer,Thrive Info Tech,https://www.naukri.com/job-listings-software-d...,https://www.ambitionbox.com/overview/thrive-in...,NaN,NaN,NaN,"Back End Developer,","Management Consulting,","Engineering - Software & QA,","Full Time, Permanent",Software Development,Teaching various computer languages and specia...,"Administration,Computer languages,C++,Analytic..."
3,210123500752,1295512,"Kolkata, Mumbai, New Delhi, Hyderabad/Secunder...",Intern,Muthoot Finance,https://www.naukri.com/job-listings-intern-mut...,https://www.ambitionbox.com/overview/muthoot-f...,1939 (84 yrs old),10k-50k,10k-50k,"Business Development Executive (BDE),","Banking,","Sales & Business Development,","Full Time, Permanent",BD / Pre Sales,Canvassing and sourcing of New Customers to in...,"Sales,Financial sector,Accounting,Gold loan,Cu..."
4,180722500200,1295512,"Kolkata, Mumbai, New Delhi, Hyderabad/Secunder...",MALE INTERN - ERODE REGION,Muthoot Finance,https://www.naukri.com/job-listings-male-inter...,https://www.ambitionbox.com/overview/muthoot-f...,1939 (84 yrs old),10k-50k,10k-50k,"Customer Service,","Banking,","Customer Success, Service & Operations,","Full Time, Permanent","Customer Success, Service & Operations - Other",Handling Walk-ins Enquiries Supporting Custome...,"Backend,Intern,Client servicing,Management"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1975,060923910646,7682,Bangalore/Bengaluru,MuleSoft Anypoint Platform Application Developer,Accenture,https://www.naukri.com/job-listings-mulesoft-a...,https://www.ambitionbox.com/overview/accenture...,1989 (34 yrs old),1 Lakh+,1 Lakh+,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1976,060923910639,7682,Bangalore/Bengaluru,Servicenow Tools Administration Program/Projec...,Accenture,https://www.naukri.com/job-listings-servicenow...,https://www.ambitionbox.com/overview/accenture...,1989 (34 yrs old),1 Lakh+,1 Lakh+,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1977,060923910635,7682,Bangalore/Bengaluru,SAP PP Production Planning & Control Discrete ...,Accenture,https://www.naukri.com/job-listings-sap-pp-pro...,https://www.ambitionbox.com/overview/accenture...,1989 (34 yrs old),1 Lakh+,1 Lakh+,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1978,060923910627,7682,Bangalore/Bengaluru,SAP Purchasing (MM PO) Technology Educator,Accenture,https://www.naukri.com/job-listings-sap-purcha...,https://www.ambitionbox.com/overview/accenture...,1989 (34 yrs old),1 Lakh+,1 Lakh+,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
finaldf['details_id'] = np.arange(finaldf.shape[0])
finaldf

,job_id,company_id,location,designation,company_name,job_url,overview_link,founded_year,india_employee_count,global_employee_count,Role:,Industry Type:,Department:,Employment Type:,Role Category:,description,skills_required,details_id
0,070923501018,19589,"Kolkata, Mumbai, New Delhi, Hyderabad/Secunder...",Associate - Inventory Accounting,Ford,https://www.naukri.com/job-listings-associate-...,https://www.ambitionbox.com/overview/ford-over...,1903 (120 yrs old),5k-10k,1 Lakh+,"Finance Executive,","Auto Components,","Finance & Accounting,","Full Time, Permanent",Finance,"Good reasoning capability, innovative mindset,...","inventory accounting,Management",0
1,270523906122?src=jobsearchDesk&sid=16941696232...,7682,"Kolkata, West Bengal, Mumbai, Maharashtra, Hyd...",CN - Comms & Media - Network Assurance - Senio...,Accenture,https://www.naukri.com/job-listings-cn-comms-m...,https://www.ambitionbox.com/overview/accenture...,1989 (34 yrs old),1 Lakh+,1 Lakh+,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,230920500882,4571516,"Kolkata, Mumbai, New Delhi, Hyderabad/Secunder...",Software Developer/Trainer,Thrive Info Tech,https://www.naukri.com/job-listings-software-d...,https://www.ambitionbox.com/overview/thrive-in...,NaN,NaN,NaN,"Back End Developer,","Management Consulting,","Engineering - Software & QA,","Full Time, Permanent",Software Development,Teaching various computer languages and specia...,"Administration,Computer languages,C++,Analytic...",2
3,210123500752,1295512,"Kolkata, Mumbai, New Delhi, Hyderabad/Secunder...",Intern,Muthoot Finance,https://www.naukri.com/job-listings-intern-mut...,https://www.ambitionbox.com/overview/muthoot-f...,1939 (84 yrs old),10k-50k,10k-50k,"Business Development Executive (BDE),","Banking,","Sales & Business Development,","Full Time, Permanent",BD / Pre Sales,Canvassing and sourcing of New Customers to in...,"Sales,Financial sector,Accounting,Gold loan,Cu...",3
4,180722500200,1295512,"Kolkata, Mumbai, New Delhi, Hyderabad/Secunder...",MALE INTERN - ERODE REGION,Muthoot Finance,https://www.naukri.com/job-listings-male-inter...,https://www.ambitionbox.com/overview/muthoot-f...,1939 (84 yrs old),10k-50k,10k-50k,"Customer Service,","Banking,","Customer Success, Service & Operations,","Full Time, Permanent","Customer Success, Service & Operations - Other",Handling Walk-ins Enquiries Supporting Custome...,"Backend,Intern,Client servicing,Management",4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1975,060923910646,7682,Bangalore/Bengaluru,MuleSoft Anypoint Platform Application Developer,Accenture,https://www.naukri.com/job-listings-mulesoft-a...,https://www.ambitionbox.com/overview/accenture...,1989 (34 yrs old),1 Lakh+,1 Lakh+,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1975
1976,060923910639,7682,Bangalore/Bengaluru,Servicenow Tools Administration Program/Projec...,Accenture,https://www.naukri.com/job-listings-servicenow...,https://www.ambitionbox.com/overview/accenture...,1989 (34 yrs old),1 Lakh+,1 Lakh+,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1976
1977,060923910635,7682,Bangalore/Bengaluru,SAP PP Production Planning & Control Discrete ...,Accenture,https://www.naukri.com/job-listings-sap-pp-pro...,https://www.ambitionbox.com/overview/accenture...,1989 (34 yrs old),1 Lakh+,1 Lakh+,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1977
1978,060923910627,7682,Bangalore/Bengaluru,SAP Purchasing (MM PO) Technology Educator,Accenture,https://www.naukri.com/job-listings-sap-purcha...,https://www.ambitionbox.com/overview/accenture...,1989 (34 yrs old),1 Lakh+,1 Lakh+,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1978


In [15]:
finaldf.columns

Index(['job_id', 'company_id', 'location', 'designation', 'company_name',
       'job_url', 'overview_link', 'founded_year', 'india_employee_count',
       'global_employee_count', 'Role:', 'Industry Type:', 'Department:',
       'Employment Type:', 'Role Category:', 'description', 'skills_required',
       'details_id'],
      dtype='object')

In [16]:
col = ['job_id', 'company_id', 'location', 'designation','details_id']
jobs_df = finaldf[col]
jobs_df.to_csv('jobs_table.csv',index = False)

col = ['company_id', 'company_name', 'founded_year', 'india_employee_count','global_employee_count']
company_df = finaldf[col]
company_df.to_csv('company_table.csv',index = False)


col = ['details_id', 'Role:', 'Industry Type:', 'Department:', 'Employment Type:', 'Role Category:', 'description', 'skills_required']
details_df = finaldf[col]
details_df.to_csv('details_table.csv',index = False)


details_df.dropna(thresh = 7).to_csv('details_new_table.csv',index = False)

In [18]:
jobs_df.to_csv('jobs_table.csv',index = False)

In [ ]:
jobs_df.to_csv()

In [20]:
jobs_df.to_csv(r"C:\Users\srish\Downloads\jobs_table.csv",index = False)

In [22]:
company_df.to_csv(r"C:\Users\srish\Downloads\company_table.csv",index = False)

In [24]:
details_df.to_csv(r"C:\Users\srish\Downloads\details_new_table.csv",index = False)

KeyError: 0